# モデルを解釈する

Azure Machine Learning を使用して、各機能が予測ラベルに与える影響の量を定量化する *Explainer* を使用して、モデルを解釈できます。一般的な Explainer は多く、それぞれ異なる種類のモデリング アルゴリズムに適しています。ただし、それらを使用する基本的なアプローチは同じです。

## SDK パッケージのインストール

このノートブックのコードを実行するには、最新バージョンの **azureml-sdk** および **azureml-widgets** パッケージに加えて、**azureml-explain-model** パッケージが必要です。また、Azure ML 解釈可能性ライブラリ (**azureml-interpret**) も使用します。これを使用すると、Azure ML 実験でトレーニングされていない場合や、Azure ML ワークスペースに登録されていない場合でも、多くの一般的な種類のモデルを解釈できます。

次のセルを実行して、これらのパッケージがインストールされていることを確認します。 

In [ ]:
!pip show azureml-explain-model azureml-interpret

## モデルを説明する

Azure Machine Learning の外部でトレーニングされたモデルから始めましょう - 下のセルを実行して、デシジョン ツリー分類モデルをトレーニングします。

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

トレーニング プロセスでは、ホールドバック検証データセットに基づいてモデル評価メトリックが生成されるため、予測の精度を把握できます。しかし、データの特徴は予測にどのような影響を与えるのでしょうか?

### モデルの説明を取得する

先にインストールした Azure ML の解釈可能性ライブラリから、モデルに適した Explainer を取得しましょう。Explainer には多くの種類があります。この例では、適切な [SHAP](https://github.com/slundberg/shap) モデル Explainer を呼び出すことによって、多くの種類のモデルを説明するために使用できる「ブラック ボックス」の説明である表形式の *Explainer* を使用します。

In [ ]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
tab_explainer = TabularExplainer(model,
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

### *グローバル*な特徴の重要度を取得する

最初に行うことは、全体的な*特徴の重要度*を評価することによってモデルを説明しようとすることです - つまり、各特徴がトレーニング データセット全体に基づいて予測に影響を与える程度を定量化します。

In [ ]:
# you can use the training data or the test data here
global_tab_explanation = tab_explainer.explain_global(X_train)

# Get the top features by importance
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

特徴の重要度が順位付けされ、最も重要な機能が最初に表示されます。

### *ローカル*な特徴の重要度を取得する

全体的な見解がありますが、個々の観察を説明はどうですか? 可能性のある各ラベル値を予測する決定に各機能が影響を与えた程度を定量化して、個々の予測の*ローカル*説明を生成しましょう。この場合、バイナリ モデルであるため、2 つのラベル (糖尿病以外と糖尿病) があります。また、データセット内の個々の観測値に対するこれらのラベル値の各特徴の影響を定量化できます。テスト データセットの最初の 2 つのケースを評価するだけです。

In [ ]:
# Get the observations we want to explain (the first two)
X_explain = X_test[0:2]

# Get predictions
predictions = model.predict(X_explain)

# Get local explanations
local_tab_explanation = tab_explainer.explain_local(X_explain)

# Get feature names and importance for each possible label
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

## モデル トレーニング実験に説明可能性を追加する

これまで見てきたように、Azure Machine Learning の外部でトレーニングされたモデルの説明を生成できます。ただし、Azure Machine Learning ワークスペースでモデルをトレーニングして登録するために実験を使用する場合は、モデルの説明を生成してログに記録できます。

次のセルでコードを実行して、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### 実験を使用してモデルをトレーニングして説明する

では、実験を作成して、必要なファイルをローカル フォルダーに配置しましょう - この場合、糖尿病データの同じ CSV ファイルを使用してモデルをトレーニングします。

In [ ]:
import os, shutil
from azureml.core import Experiment

# Create a folder for the experiment files
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

次の特徴を含む以外 Azure ML トレーニング スクリプトと同様のトレーニング スクリプトを作成します。

- 以前使用したモデルの説明を生成する同じライブラリがインポートされ、グローバルな説明を生成するために使用されます
- **ExplanationClient** ライブラリを使用して、説明を実験出力にアップロードします

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Import Azure ML run library
from azureml.core.run import Run

# Import libraries for model explanation
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# Separate features and labels
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# Get explanation
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# Complete the run
run.complete()

実験にはスクリプトを実行するための Python 環境が必要なため、そのための Conda 仕様を定義します。トレーニング環境には **azureml-interpret** ライブラリが含まれているので、スクリプトは **TabularExplainer** を作成して **ExplainerClient** クラスを使用できる点に留意してください。

In [ ]:
%%writefile $experiment_folder/interpret_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- pip
- pip:
  - azureml-defaults
  - azureml-interpret

これで実験を実行できます。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails


# Create a Python environment for the experiment
explain_env = Environment.from_conda_specification("explain_env", experiment_folder + "/interpret_env.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                      script='diabetes_training.py',
                      environment=explain_env) 

# submit the experiment
experiment_name = 'mslearn-diabetes-explain'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

## 特徴の重要度の値を取得する

実験の実行が完了したら、**ExplanationClient** クラスを使用して、実行用に登録された説明から特徴の重要度を取得できます。

In [ ]:
from azureml.interpret import ExplanationClient

# Get the feature explanations
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Azure Machine Learning Studio でモデルの説明を表示する

また、実行の詳細ウィジェットの**実行の詳細を表示**リンクをクリックすると、Azure Machine Learning Studio の実行が表示され、**説明**タブを表示できます。次に以下を実行します。

1. 表形式の Explainer の説明 ID を選択します。
2. 全体的なグローバル特徴の重要度を示す**特徴の重要度の集計**グラフを表示します。
3. テスト データの各データ ポイントを示す**個別の特徴の重要度**グラフを表示します。
4. 個々のポイントを選択すると、選択したデータ ポイントの個々の予測のローカル特徴の重要度が表示されます。
5. 「**新しいコホート**」 ボタンを使って、次の設定でデータのサブセットを定義します。
    - **データセット コホート名**：25歳未満
    - **フィルターを選択する**: データセット
        - 25歳未満（新しいコホートを保存する前に、必ずこのフィルターを追加する）。
6. 25歳以上の年齢フィルターを使用して、「**25歳以上**」 という名前の 2 つ目のコホートを作成します。
6. **集計機能の重要度**の視覚化を確認し、定義した 2 つのコホートの相対的な機能の重要度を比較します。コホートを比較する能力により、データ母集団の複数のサブセットについて、機能が予測にどのような影響を与えるかを確認できます。



**詳細情報**: Azure ML での Explainer の使用の詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability)を参照してください。 